In [1]:
#import modules
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
#get files from cleaned data
players = pd.read_csv('players.csv', index_col='Unnamed: 0')
coaches = pd.read_csv('coaches.csv', index_col='Unnamed: 0')
champions = pd.read_csv('champions.csv', index_col='Unnamed: 0')

In [3]:
print(players.shape)
players.head()

(6371, 32)


,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,TRB,AST,STL,BLK,TOV,PF,PTS,Year,Champ,Runner-Up
0,Arron Afflalo,SG,24.0,DEN,82.0,75.0,27.1,3.3,7.1,0.465,...,3.1,1.7,0.6,0.4,0.9,2.7,8.8,2010,0,0
1,Alexis Ajinça,C,21.0,CHA,6.0,0.0,5.0,0.8,1.7,0.500,...,0.7,0.0,0.2,0.2,0.3,0.8,1.7,2010,0,0
2,LaMarcus Aldridge,PF,24.0,POR,78.0,78.0,37.5,7.4,15.0,0.495,...,8.0,2.1,0.9,0.6,1.3,3.0,17.9,2010,0,0
3,Joe Alexander,SF,23.0,CHI,8.0,0.0,3.6,0.1,0.8,0.167,...,0.6,0.3,0.1,0.1,0.0,1.1,0.5,2010,0,0
4,Malik Allen,PF,31.0,DEN,51.0,3.0,8.9,0.9,2.3,0.397,...,1.6,0.3,0.2,0.1,0.4,1.3,2.1,2010,0,0


In [4]:
print(coaches.shape)
coaches.head()

(332, 26)


,Coach,Team,F-Seasons,Car-Seasons,CR-G,CR-W,CR-L,FR-G,FR-W,FR-L,...,CP-L,FP-G,FP-W,FP-L,Car.P-G,Car.P-W,Car.P-L,Year,Champ,Runner-Up
0,Mike Woodson,ATL,6,6,82,53,29,492,206,286,...,7.0,29.0,11.0,18.0,29.0,11.0,18.0,2010,0,0
1,Doc Rivers,BOS,6,11,82,50,32,492,280,212,...,9.0,71.0,41.0,30.0,86.0,46.0,40.0,2010,0,1
2,Larry Brown,CHA,2,29,82,44,38,164,79,85,...,4.0,4.0,0.0,4.0,235.0,120.0,115.0,2010,0,0
3,Vinny Del Negro,CHI,2,2,82,41,41,164,82,82,...,4.0,12.0,4.0,8.0,12.0,4.0,8.0,2010,0,0
4,Mike Brown,CLE,5,5,82,61,21,410,272,138,...,5.0,71.0,42.0,29.0,71.0,42.0,29.0,2010,0,0


In [5]:
#Descriptive statistics
players.columns

Index(['Player', 'Pos', 'Age', 'Tm', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P',
       '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB',
       'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'Year', 'Champ',
       'Runner-Up'],
      dtype='object')

In [6]:
#list of numerical and categorical features
player_num_features = ['Age', 'G', 'GS','MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', '2P', '2PA',                         '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK',                        'TOV', 'PF','PTS']
player_cat_features = ['Player', 'Pos', 'Tm', 'Year', 'Champ', 'Runner-Up']

# Players 

## Exploring Null Values:
- There are players who are on the team, but haven't played games
- Some players entered a game, but didn't record a shot (minutes played != 0)
- There are players who won't have attempted two pointers or three pointers e.g. centers, some forwards
- Those who don't get to the free throw line

In [7]:
def get_null_columns(df):
    nulls = df.isnull().sum()
    for key, i in zip(list(nulls.keys()), nulls):
        if i > 0: print(key, '\t', i)

In [8]:
get_null_columns(players)

Age 	 237
G 	 237
GS 	 237
MP 	 237
FG 	 237
FGA 	 237
FG% 	 271
3P 	 237
3PA 	 237
3P% 	 1026
2P 	 237
2PA 	 237
2P% 	 309
eFG% 	 271
FT 	 237
FTA 	 237
FT% 	 540
ORB 	 237
DRB 	 237
TRB 	 237
AST 	 237
STL 	 237
BLK 	 237
TOV 	 237
PF 	 237
PTS 	 237


Note, the FG% and eFG% are missing 34 values --> these players did not attempt a shot, whether or not they played. Let's see if they played. Other observations:
- 237 players were rostered, but did not play any minutes of basketball for their teams
- 271-237 players recorded game minutes, but never attempted a shot in a game 
- 1026-237 players recorded minutes, but didn't attempt a three pointer
- 309-237 players recorded minutes, but didn't attempt a two pointer
- 540-237 players recorded minutes, but didn't attempt a free throw


### What to remove?
- We don't want to keep any players who didn't play any minutes (G == 0 or MPG == 0.0)
- Might keep players whose average minutes are 0 because maybe they played a game or two. Have to see first.

In [9]:
no_min = len(players[players['MP'].isnull()])
print('Players with 0 minutes played are {}% of the data'.format(round(no_min*100/len(players), 2)))

#drop this population -> cannot extract info to answer business question
played_minutes = players[players['MP'].isnull() == False]
played_size = len(played_minutes)
print('We have gone from {} to {} observations.'.format(len(players), len(played_minutes)))

Players with 0 minutes played are 3.72% of the data
We have gone from 6371 to 6134 observations.


### Exploring small impact players
Some players have minutes recorded, but they could be entering the game and not attempting shots/assists or affecting the defensive effort with rebounds/steals/blocks. Using a threshold of MP = 2.0/per game and 5 or less games to start to see if we should keep or drop these players.

In [10]:
played_few_mins = played_minutes[(played_minutes['MP'] <= 2.0) & (played_minutes['G'] <= 6)
                    & (played_minutes['PTS'] <= 2.0)]
pct_total = len(played_few_mins)*100/len(played_minutes)
print('Players who had small impact are {}% of the data'.format(round(pct_total,2)))
played_few_mins[player_num_features].describe()

Players who had small impact are 0.95% of the data


,Age,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
count,58.000000,58.000000,58.0,58.000000,58.000000,58.000000,37.000000,58.000000,58.000000,12.000000,...,6.000000,58.000000,58.000000,58.000000,58.000000,58.000000,58.000000,58.000000,58.000000,58.000000
mean,24.293103,2.258621,0.0,1.517241,0.160345,0.525862,0.312622,0.017241,0.108621,0.166667,...,0.458333,0.122414,0.215517,0.336207,0.058621,0.027586,0.034483,0.103448,0.191379,0.372414
std,2.828481,1.318696,0.0,0.539375,0.293162,0.484337,0.377660,0.099363,0.248705,0.389249,...,0.332290,0.308956,0.320527,0.441567,0.140218,0.091370,0.105218,0.220816,0.462368,0.642034
min,20.000000,1.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,22.250000,1.000000,0.0,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.312500,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,23.500000,2.000000,0.0,1.750000,0.000000,0.500000,0.000000,0.000000,0.000000,0.000000,...,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,25.750000,3.000000,0.0,2.000000,0.300000,1.000000,0.500000,0.000000,0.000000,0.000000,...,0.500000,0.000000,0.300000,0.650000,0.000000,0.000000,0.000000,0.150000,0.000000,0.700000
max,32.000000,6.000000,0.0,2.000000,1.000000,2.000000,1.000000,0.700000,1.000000,1.000000,...,1.000000,1.500000,1.000000,1.500000,0.700000,0.400000,0.500000,1.300000,2.000000,2.000000


Of these players:
- The only statistic that sticks out is number of games played. There is an outlier of 9 games over the 75% percentile. There is also a player(s) who averaged 3.0/game which is also way over the 75% percentile value of 0.7/game
- The standard deviations for game metrics are huge which makes sense because the chances of these individuals playing and their impact on a game are stochastic, and cannot be gainfully analysed/all over the place.
- After filtering out the highest number of games (6 - consistency check) and the highest point average (2.0/game), the rest of the players' impact seems minimal.
**Decision:** Dropping them from the dataset (58 observations -> .95% of dataset)

In [11]:
played_minutes = played_minutes[(played_minutes['MP'] > 2.0) | (played_minutes['G'] > 6)
                    | (played_minutes['PTS'] > 2.0)]
print('We have now gone from {} to {} observations.'.format(played_size, len(played_minutes)))

We have now gone from 6134 to 6076 observations.


In [12]:
#rechecking null values
get_null_columns(played_minutes)

FG% 	 13
3P% 	 743
2P% 	 46
eFG% 	 13
FT% 	 251


# Coaches 

In [13]:
coaches.columns.values

array(['Coach', 'Team', 'F-Seasons', 'Car-Seasons', 'CR-G', 'CR-W',
       'CR-L', 'FR-G', 'FR-W', 'FR-L', 'Car-G', 'Car-W', 'Car-L',
       'Car-W%', 'CP-G', 'CP-W', 'CP-L', 'FP-G', 'FP-W', 'FP-L',
       'Car.P-G', 'Car.P-W', 'Car.P-L', 'Year', 'Champ', 'Runner-Up'],
      dtype=object)

In [14]:
get_null_columns(coaches)
#recall, coaches table key:

CP-G 	 172
CP-W 	 172
CP-L 	 172
FP-G 	 114
FP-W 	 114
FP-L 	 114
Car.P-G 	 65
Car.P-W 	 65
Car.P-L 	 65


Coaches table is straight forward and no trimming will help to extract more data about finals appearances. It will be left as is. The next steps involved extracting data needed to analyze finals and championship teams.

In [16]:
#trimmed data saved
played_minutes.to_csv('players_trim.csv')